In [39]:
import chromadb
from chromadb.utils import embedding_functions

In [41]:
chroma_client = chromadb.Client()
client_persistent = chromadb.PersistentClient(path='/home/egarces/chatbot/databases/embeddings-public')

In [42]:
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [44]:
client_persistent.list_collections()

[Collection(id=2157830b-58d1-4df4-af31-20a1e593da24, name=langchain)]

In [45]:
db = client_persistent.get_or_create_collection(name='langchain', embedding_function=sentence_transformer_ef)

In [46]:
db.peek(1)

{'ids': ['2d23ddd2-e2d6-11ee-b3e0-b9320b25b112'],
 'embeddings': array([[-5.15644476e-02, -5.58340317e-03,  9.14661735e-02,
          7.76682124e-02,  2.59677142e-01,  3.59709598e-02,
          2.96066813e-02,  9.95760933e-02, -1.91513687e-01,
         -6.49192482e-02,  7.39299580e-02, -2.54625320e-01,
          1.87229231e-01,  3.27701159e-02, -3.77930626e-02,
         -5.52089997e-02,  7.65995979e-02, -6.35847375e-02,
         -5.15189394e-02,  8.01788494e-02,  1.56739667e-01,
         -1.91984907e-01, -7.16559067e-02,  2.28334770e-01,
         -1.95928171e-01, -2.25191507e-02, -1.55682176e-01,
          6.61088377e-02,  1.07859507e-01, -1.79414764e-01,
         -9.47954059e-02,  1.68424815e-01, -2.30620250e-01,
         -2.81535357e-01,  1.59576550e-01, -8.16967487e-02,
         -2.04534624e-02,  1.67793185e-02, -1.68097749e-01,
         -9.94877983e-03,  3.52992527e-02, -7.71250203e-02,
          1.42698422e-01, -1.86311025e-02, -1.77909419e-01,
         -8.91768560e-02,  1.1319543

In [89]:
# delete info by element

element_id = 'ElementId'
db.delete(where={"source": {"$eq": element_id}})

In [27]:
#add info by element based


""" 
JSON Structure

{
    "source" : ElementId,
    "document" : "la historia de la vida"  # relevant info
    "keywords": ["word1", "word2"]
}

"""



# documents = ["Este es el tercer documento "]
# metadatas = [
#     {"source": element_id}
# ]

# # Genera IDs automáticamente
# ids = [str(uuid.uuid4()) for _ in documents]

# # Agrega los documentos a la colección
# db.add(ids=ids, documents=documents, metadatas=metadatas)



In [8]:
from sentence_transformers import SentenceTransformer, util

# Configuración del modelo de Sentence Transformers
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [9]:
# Ejemplo de JSON
data = {
    "source": "ElementId",
    "document": "la historia de la vida es larga y llena de eventos. A través del tiempo, muchas culturas han dejado huellas en la historia...",  # texto de ejemplo
    "keywords": ["historia", "vida", "cultura"]
}

In [ ]:
import json
import uuid
from transformers import AutoTokenizer


def split_text_by_tokens_transformers(text, max_tokens, overlap=0, model_name="sentence-transformers/all-MiniLM-L6-v2"):
    """
    Divide un texto en fragmentos basados en tokens, compatible con modelos de sentence-transformers.

    Args:
        text (str): Texto a dividir.
        max_tokens (int): Número máximo de tokens por fragmento.
        overlap (int): Número de tokens solapados entre fragmentos.
        model_name (str): Modelo de transformers para determinar el tokenizador.

    Returns:
        List[str]: Lista de fragmentos de texto.
    """
    # Validar el solapamiento
    if overlap >= max_tokens:
        raise ValueError("El solapamiento debe ser menor que el número máximo de tokens.")
    
    # Inicializar el tokenizador
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Tokenizar el texto (convertir a lista de tokens)
    encoded_input = tokenizer(text, truncation=False, padding=False, add_special_tokens=False)
    
    # Verificar si la tokenización fue exitosa
    if "input_ids" not in encoded_input:
        raise ValueError("La tokenización ha fallado, no se generaron 'input_ids'.")
    
    tokens = encoded_input["input_ids"]
    
    # Si el texto es más pequeño que el máximo de tokens, no es necesario dividir
    if len(tokens) <= max_tokens:
        return [tokenizer.decode(tokens, skip_special_tokens=True)]
    
    # Dividir en fragmentos
    chunks = []
    start = 0
    while start < len(tokens):
        # Si estamos en el último fragmento, no hay necesidad de avanzar más allá de los tokens disponibles
        end = min(start + max_tokens, len(tokens))
        chunk_tokens = tokens[start:end]
        chunk_text = tokenizer.decode(chunk_tokens, skip_special_tokens=True)
        chunks.append(chunk_text)
        
        # Si no hay suficiente espacio para el solapamiento, terminamos
        if end == len(tokens):
            break
        
        start = end - overlap  # Retroceder para el solapamiento
    return chunks



# Función para crear el JSON particionado
def partition_json(data, max_tokens=20, overlap_tokens=2):
    # Particionar el texto en fragmentos

    keywords_unificado = "  ".join(data["keywords"])

    # Inicializar el tokenizador
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
    
    # Tokenizar el texto (convertir a lista de tokens)
    encoded_input = tokenizer(keywords_unificado, truncation=False, padding=False, add_special_tokens=False)
    
    # Verificar si la tokenización fue exitosa
    if "input_ids" not in encoded_input:
        raise ValueError("La tokenización ha fallado, no se generaron 'input_ids'.")
    
    tokens = encoded_input["input_ids"]

    partitions = split_text_by_tokens_transformers(data["document"], max_tokens-len(tokens), overlap_tokens)
    #partitions=['prueba 1', 'prueba 2']#data["document"]
    #print(partitions)
    # # Crear el nuevo JSON con los fragmentos
    partitioned_data = []
    for i, partition in enumerate(partitions):

        #print(partition)
        partitioned_data.append({
            "source": data["source"],
            "document": partition + ' ' + keywords_unificado,  # Reconstruir el fragmento de texto
            "keywords": data["keywords"],
        })
    
    return partitioned_data

# Ejemplo de datos originales
data = {
    "source": "ElementId",
    "document": "Este es un ejemplo de texto que queremos dividir en fragmentos basados en tokens. El solapamiento asegura que haya contexto compartido entre los fragmentos. Es una forma útil de preprocesar texto para modelos como los de sentence-transformers.",
    "keywords": ["word1", "word2"]
}

# Particionar el JSON
partitioned_json = partition_json(data)
#print(partitioned_json)

# Imprimir el JSON particionado
#print(json.dumps(partitioned_json))
# Generar la lista formateada
sources = [{"source": item['source']} for item in partitioned_json]


documents = [item['document'] for item in partitioned_json]

# Genera IDs automáticamente
ids = [str(uuid.uuid4()) for _ in documents]


# Imprimir el resultado
print(sources)
print(documents)
print(ids)

# Agrega los documentos a la colección
db.add(ids=ids, documents=documents, metadatas=sources)







[{'source': 'ElementId'}, {'source': 'ElementId'}, {'source': 'ElementId'}, {'source': 'ElementId'}, {'source': 'ElementId'}]
['este es un ejemplo de texto que queremos divid word1  word2', 'dividir en fragmentos basados en tokens. el solapamiento word1  word2', '##apamiento asegura que haya contexto compartido entre word1  word2', 'entre los fragmentos. es una forma util de preproces word1  word2', '##rocesar texto para modelos como los de sentence - transformers. word1  word2']
['01022da5-df88-4eff-baf3-91856ff80110', 'c3bf9d1d-030a-4fcf-bde1-1b08b4020471', '4c0b0650-36e7-4552-852b-7daedb4bd45e', '12b8a199-a2cb-467e-a6d5-fac25a74c3e8', '99259091-be1d-4b44-a3ae-ac0b1265ec72']


[]

In [92]:
#List by elementId 


resultados = db.query(
    query_texts=[""], 
    where={
        "source": {"$eq": 'ElementId'},
    },
    n_results=10  # Especifica el número de resultados que deseas obtener
)
resultados

{'ids': [['12b8a199-a2cb-467e-a6d5-fac25a74c3e8',
   '01022da5-df88-4eff-baf3-91856ff80110',
   'c3bf9d1d-030a-4fcf-bde1-1b08b4020471',
   '4c0b0650-36e7-4552-852b-7daedb4bd45e',
   '99259091-be1d-4b44-a3ae-ac0b1265ec72']],
 'embeddings': None,
 'documents': [['entre los fragmentos. es una forma util de preproces word1  word2',
   'este es un ejemplo de texto que queremos divid word1  word2',
   'dividir en fragmentos basados en tokens. el solapamiento word1  word2',
   '##apamiento asegura que haya contexto compartido entre word1  word2',
   '##rocesar texto para modelos como los de sentence - transformers. word1  word2']],
 'uris': None,
 'data': None,
 'metadatas': [[{'source': 'ElementId'},
   {'source': 'ElementId'},
   {'source': 'ElementId'},
   {'source': 'ElementId'},
   {'source': 'ElementId'}]],
 'distances': [[23.348649631292297,
   26.09077413200531,
   28.330692514083804,
   28.996867100705746,
   30.45221425930692]],
 'included': [<IncludeEnum.distances: 'distances'>,
  